In [1]:
from openai import OpenAI
import numpy as np
import json
import faiss

In [2]:
#load json files
def load_documents(json_file):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)
        
    return data

# Generate embeddings for each document
def get_embedding(text):
    response = client.embeddings.create(model = "text-embedding-ada-002", input = text)
    embedding_array = np.array(response.data[0].embedding)
    
    return embedding_array


In [3]:
with open("C:/Users/rshua/keys/secret_key.json", "r") as file:
    secrets = json.load(file)
    api_key = secrets["OPENAI_API_KEY"]

client = OpenAI(api_key = api_key)

In [4]:
# data = load_documents("C:/Users/rshua/fantasy-football-chatbot/nfl_season_stats_partial.json")
data = load_documents("nfl_season_stats_partial.json")

In [5]:
# Convert JSON to a list of text documents
documents = []

for week_data in data["Weeks"]:
    week = week_data["week"]
    for position, position_data in week_data["Positions"].items():
        for player_name, player_stats in position_data["Players"].items():
            text = f"Week: {week}, Position: {position}, Player: {player_name}, Team: {player_stats['team']}, "
            text += f"Dropbacks: {player_stats['dropbacks']}, Attempts: {player_stats['att']}, Completions: {player_stats['comp']}, "
            text += f"Yards: {player_stats['yds']}, TDs: {player_stats['tds']}, INTs: {player_stats['ints']}, "
            text += f"Sacks: {player_stats['sks']}, Rush Carries: {player_stats['rushCarries']}, Rush Yards: {player_stats['rushYds']}, "
            text += f"Rush TDs: {player_stats['rushTds']}, Fumbles: {player_stats['fumbles']}, Fantasy Points: {player_stats['fantasyPts']}"
            documents.append({"text": text})
            # print(week)

last_week = week
next_week = week + 1


In [6]:
# Create a list of text documents and their embeddings
# texts = [doc["text"] for doc in documents]
texts = []

for i in documents:
    text_content = i["text"]
    texts.append(text_content)

embeddings = np.array([get_embedding(text) for text in texts])

# Store embeddings in FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# dimension = embeddings.shape[1]
# index = faiss.IndexFlatL2(dimension)
# index.add(np.array([embeddings]))
# faiss.write_index(index, "vector_store.index")



In [10]:
# def retrieve_relevant_docs(query):
#     query_embedding = get_embedding(query)
#     query_embedding = query_embedding.reshape(1, -1)

#     distances, indices = index.search(query_embedding)

#     relevant_docs = []

#     for i in indices[0]:
#         relevant_docs.append(texts[i])
    
#     return relevant_docs

In [7]:
# Function to retrieve the most relevant documents
def retrieve_relevant_docs(query, top_k = 6):
    query_embedding = get_embedding(query)
    query_embedding = query_embedding.reshape(1, -1)

    distances, indices = index.search(query_embedding, top_k)

    relevant_docs = []

    for i in indices[0]:
        relevant_docs.append(texts[i])

    return relevant_docs


    # query_embedding = get_embedding(query)
    # distances, indices = index.search(query_embedding.reshape(1, -1), top_k)
    
    # return [texts[i] for i in indices[0]]



In [37]:
system_message = """
### Role & Purpose
You are an expert Fantasy Football AI assistant designed to provide data-driven insights, strategic advice, and statistical analysis for NFL player performance. Your responses must be:
- **Concise** but informative.
- **Factual** (grounded in retrieved data).
- **Actionable** (offer clear recommendations when asked).

---

### Core Responsibilities
1. **Statistical Summaries**
   - Always specify the week number when referencing data (e.g., "In Week 7, Player X had...").
   - If no data exists for a requested week/player, state: "No records found for [Player] in Week [X]."

2. **Lineup Advice (Start/Sit)**
   - Compare players using:
     - Recent performance (last 2-3 weeks)
     - Matchup difficulty (opposing defense strength)
     - Volume trends (targets/carries)
   - Example: "Start Player A over Player B due to consistent red-zone targets."

   - **Response Format:**
    [Player A] vs [Player B] (Week {X}):
    ▸ **Recent**: [A-PTS] vs [B-PTS] (Last 3 avg)
    ▸ **Matchup**: [A-DEF] (Rank) vs [B-DEF] (Rank)
    ▸ **Volume**: [A-TGT]/[A-CAR] vs [B-TGT]/[B-CAR]
    ---
    [Verdict]: "Start [Player] because [reason]."
    [Risk]: "[Potential downside]."

3. **Trade Recommendations**
   - Use terms like "Buy low" or "Sell high" based on value trends.
   - Highlight under/overperforming players with reasoning.

4. **Next-Week Projections (Week {next_week})**
   - Base predictions on:
     - Recent form (last 3 weeks)
     - Matchup context
     - Role changes (injuries, depth chart)
   - Include estimates for:
     - Passing: Yards, TDs, INTs
     - Rushing: Yards, TDs
     - Fantasy points (specify scoring format)

---

### Key Rules
- **Temporal Context:**
  - "Last week" = Week {last_week}, "Next week" = Week {next_week}.
  - Reject requests beyond Week {next_week}: "I can only project for Week {next_week}."

- **Data Integrity:**
  - Never guess stats. If unsure: "Insufficient data to answer."
  - For rookies/injured players: "Limited sample size; monitor practice reports."

- **Response Format:**
[Week X] [Player Name]:

Passing: [YDS]/[TDs]/[INTs]

Rushing: [YDS]/[TDs]

Fantasy: [PTS] (PPR/Standard)

[Insight]: "[Trend/Matchup Analysis]."
[Advice]: "[Start/Sit/Trade] because [reason]."
"""

# system_message = (
#     "You are a helpful Fantasy Football assistant specializing in providing fantasy football advice. You assist with:"
#     "Summarizing NFL player statistics based on retrieved information"
#     "Providing valuable line up suggestions like start/sit recommendations"
#     "Providing insightful trade advice"

#     "If asked about information from a specific week, only retrieve documents from that week and always mention the specific week number in your response.\n"

#     f"'Last week' refers to Week {last_week} of the season, and 'next week' refers to Week {next_week}.\n"

#     "If asked to predict performance for 'next week', base your prediction on the player's skill level, "
#     "recent performances, and overall season trends.\n"

#     "Your 'next week' predictions should include key metrics such as passing yards, passing touchdowns, rushing yards, "
#     "and total fantasy points. Ensure your estimates are reasonable based on past data.\n"
# )

In [39]:
user_query = "Next week, would you start Jalen Hurts or BakerMayfield?"

retrieved_docs = retrieve_relevant_docs(user_query)
context = "\n".join(retrieved_docs)


In [40]:
# print(f"Retrieved Context:\n{context}")
# print("")
# print(retrieved_docs)

In [41]:
prompt = f"Use the following information to answer the query:\n\n{context}\n\nUser Query: {user_query}\n\nAnswer:"

completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
)

print(completion.choices[0].message.content)


To provide an informed comparison for Week 9 between Jalen Hurts and Baker Mayfield, I will summarize Hurts' recent performances (focusing on the last 3 weeks) and I need comparative data for Mayfield, which isn't currently provided. Here’s the analysis for Jalen Hurts:

- **Week 6**: Fantasy Points: 21.9
- **Week 7**: Fantasy Points: 22.8
- **Week 8**: Fantasy Points: 35.1

Given Jalen Hurts's impressive performance in Week 8 and steady points in previous weeks, he appears to be in good form. His dual-threat capability with substantial rushing yards and touchdowns adds a consistent upside to his fantasy value.

### Recommendation:
Since I lack data on Baker Mayfield's recent performances, specific matchup information, and his role in his team's offense recently, my advice leans towards starting **Jalen Hurts** in Week 9. His ability to accumulate fantasy points both in the air and on the ground, shown especially in Week 8, makes him a strong candidate. However, consider updating with 

In [14]:
# completion = client.chat.completions.create(
#   model="gpt-4o-mini",
#   store=True,
#   messages=[
#     {"role": "user", "content": "How did Saquon Barkley do in the 2024 season?"}
#   ]
# )

# print(completion.choices[0].message);
